In [1]:
import re
import pandas as pd
from openpyxl import load_workbook
pd.options.display.max_rows = None

In [34]:
# 開啟檔案
def open_brd_file(filepath):
    f = open(filepath)
    return f

#用空格拆分每一行
def splitRow(string):
    string = string.split()
    return string

#刪除|開頭
def deleteRow1(string):
    pattern = re.compile(r"\|")
    delete = pattern.findall(string)
    return delete
#刪除- - -開頭
def deleteRow2(string):
    pattern = re.compile(r"^\-$")
    delete = pattern.findall(string)
    return delete

#找到D開頭
def findD(string):
    pattern = re.compile(r"^D")
    locationD = pattern.findall(string)
    return locationD

# 存excel
def save_excel(write, df, sheetName):
#     df = pd.DataFrame(datalist)
    df.to_excel(write, sheet_name=f'{sheetName}', index=False)
    
#刪多餘的raw
def deleteNullVIAROW(df):
    for i in range(df.shape[0]):
        if df.loc[i, "location"] == "VIA" and df.loc[i, "gap"] == 0.0:
            df.drop(i, axis=0, inplace=True)
            
#這一份txt total的path 數量
def branchPathNum(df):
    #扣掉net name/start_end/total length
    maxPathNum = int((df.shape[1] - 7) / 2)
    return maxPathNum

In [22]:
#step1: 開啟檔案
filepath = r"0511.txt"
f = open_brd_file(filepath)
print(f"step1: Open file. (File name: {filepath}) -DONE")

#step2: 刪除不必要行數
rawData = []
SQS = []
summary = []
netNameList = []

for i in f:
    i = i.replace("\n","")#去掉換行符號
    row = splitRow(i)
#     print(row)
    pattern = re.compile(r"^D")
    delete = pattern.findall(row[0])
    if delete:
        print(i)
        print(delete)

step1: Open file. (File name: 0511.txt) -DONE
                   D12304.2*       6004.250 5894.975 L 5170.361  BOTTOM
['D']
                   D3701.2         6530.000 4706.400 L  201.121  TOP
['D']
                   D3702.2         6489.000 4706.400 L  242.121  TOP
['D']
                   D3702.1*        6489.000 4669.000 L 1593.911  TOP
['D']
                   D8801.2         2232.000 3638.000 L  379.811  BOTTOM
['D']
                   D5105.1*        8978.000 5905.000 L    0.000  
['D']
                   D12302.2*       5820.275 5766.925 L 2876.545  BOTTOM
['D']
                   D1301.1*        6245.000 3673.700 L  766.103  TOP
['D']
                   D5903.1         2447.175 4426.325 L 4097.559  TOP
['D']
                   D6401.2         8580.000  351.600 L  135.640  TOP
['D']
                   D3701.1*        6530.000 4669.000 L 7689.236  TOP
['D']
                   D5903.2*        2447.175 4463.725 L 2087.974  TOP
['D']
                   D5102.2         8367.000 6080

In [36]:
#step1: 開啟檔案
filepath = r"0511.txt"
f = open_brd_file(filepath)
print(f"step1: Open file. (File name: {filepath}) -DONE")

#step2: 刪除不必要行數
rawData = []
SQS = []
summary = []
netNameList = []

for i in f:
    i = i.replace("\n","")#去掉換行符號
    row = splitRow(i)

    if not deleteRow1(row[0]):
        if not deleteRow2(row[0]):
            rawData.append(row)
print(f"step2: Clear excess rows. -DONE")

#step3: 整理成需要的格式
for i in rawData:
#     if len(i) == 1: #netname那一行

    if len(i) < 3: #netname那一行 ######################### TODO這行應該要改
        data = {} #清空data
        netName = i[0]
        
        netNameList.append(netName) #建立netnamelist
    elif i[-1] == "0.000": #找到location
#         print(i)
        data = {} #清空
        
        D = findD(i[0])
        if not D:
            
            try:
                location, length = i[0].replace("*", ""), float(i[-1])
                data.update({"net_name" : netName, "location" : location, "length": length, "layer": ""})
            except:
                print(netName," has error.")
        
            SQS.append(data)
    elif i[-1] == "mils": #找到TOTAL
        data = {} #清空
        ttlLength = float(i[3])
        data.update({"net_name" : netName, "total_length" : ttlLength})
        summary.append(data)
        
    else:
        data = {} #清空
        
        D = findD(i[0])
        if not D:
            
            try:
                location, length, layer = i[0].replace("*", ""), float(i[-2]), i[-1]
                data.update({"net_name" : netName, "location" : location, "length": length, "layer": layer})
            except:
                print(netName, "has error.")

            SQS.append(data)

#存取df
dfsummary, dfSQS, dfSQSR = pd.DataFrame(summary), pd.DataFrame(SQS), pd.DataFrame(SQS)
print("step3: Transfer file format. -DONE")

step1: Open file. (File name: 0511.txt) -DONE
step2: Clear excess rows. -DONE
step3: Transfer file format. -DONE


In [37]:
dfSQS

,net_name,location,length,layer
0,0401,JRJ1.A2,0.000,
1,0401,VIA,1679.727,IN1
2,0401,C7501.1,1984.400,TOP
3,0401,R7501.2,2018.400,TOP
4,0402,JRJ1.B2,0.000,
5,0402,R7502.2,254.444,TOP
6,0402,VIA,294.444,TOP
7,7302,R7310.2,0.000,
8,7302,U7301.33,79.790,TOP
9,7303,U7301.35,0.000,


In [8]:
for i in rawData:
    print(i)

['0401']
['JRJ1.A2*', '457.283', '732.282', 'L', '0.000']
['VIA', '819.572', '1076.287', 'V', '1679.727', 'IN1']
['C7501.1', '710.000', '817.000', 'L', '1984.400', 'TOP']
['R7501.2*', '710.000', '783.000', 'L', '2018.400', 'TOP']
['TOTAL', '1', 'VIA(S)', '2018.400', 'mils']
['0402']
['JRJ1.B2*', '457.283', '181.102', 'L', '0.000']
['R7502.2', '701.000', '207.000', 'L', '254.444', 'TOP']
['VIA*', '701.000', '167.000', 'V', '294.444', 'TOP']
['TOTAL', '1', 'VIA(S)', '294.444', 'mils']
['7302']
['R7310.2*', '2120.000', '1921.000', 'L', '0.000']
['U7301.33*', '2082.770', '1975.410', 'L', '79.790', 'TOP']
['TOTAL', '0', 'VIA(S)', '79.790', 'mils']
['7303']
['U7301.35*', '2114.260', '1975.410', 'L', '0.000']
['R7311.2', '2145.000', '1921.000', 'L', '67.143', 'TOP']
['VIA*', '2184.000', '1921.000', 'V', '106.143', 'TOP']
['TOTAL', '1', 'VIA(S)', '106.143', 'mils']
['7304']
['Y7301.3*', '1908.280', '2343.210', 'L', '0.000']
['C7314.2', '1911.500', '2308.000', 'L', '36.544', 'TOP']
['U7301.10*'

['C3908.1', '6772.450', '3918.625', 'L', '189.048', 'TOP']
['R3903.1', '6775.450', '3896.400', 'L', '212.516', 'TOP']
['Y3901.1*', '6793.010', '3863.200', 'L', '252.990', 'TOP']
['TOTAL', '1', 'VIA(S)', '252.990', 'mils']
['27MHZ_OUT']
['UGPU1.B10*', '6809.448', '4019.295', 'D', '0.000']
['VIA(T)', '6793.628', '4002.900', 'V', '22.948', 'BOTTOM']
['R3903.2*', '6798.450', '3896.400', 'L', '143.810', 'TOP']
['VIA', '6740.075', '3937.700', 'V', '294.106', 'BOTTOM']
['R3904.1*', '6725.350', '3917.150', 'L', '320.755', 'TOP']
['TOTAL', '2', 'VIA(S)', '320.755', 'mils']
['ACIN']
['R8544.2*', '2385.000', '4339.000', 'L', '0.000']
['D8501.1', '2434.000', '4308.000', 'L', '61.841', 'TOP']
['VIA', '2336.300', '4047.900', 'V', '389.011', 'TOP']
['VIA', '8826.850', '2592.275', 'V', '10345.758', 'IN1']
['PR0223.1*', '8834.850', '2563.725', 'L', '10377.622', 'TOP']
['TOTAL', '2', 'VIA(S)', '10377.62', 'mils']
['ACIN_EC']
['D8501.2*', '2434.000', '4223.750', 'L', '0.000']
['R8543.1', '2385.000', '427

['JDIMM1.174*', '4021.062', '2213.072', 'D', '0.000']
['VIA', '4001.045', '2285.700', 'V', '80.919', 'BOTTOM']
['VIA', '4299.855', '3683.385', 'V', '1651.190', 'IN3']
['UCPU1.AF50*', '4351.751', '3797.835', 'L', '1787.136', 'BOTTOM']
['TOTAL', '2', 'VIA(S)', '1787.136', 'mils']
['DDR1_DQ1_6']
['JDIMM1.173*', '4011.220', '2535.906', 'D', '0.000']
['VIA', '3985.715', '2408.525', 'V', '137.960', 'BOTTOM']
['VIA', '4300.015', '3636.415', 'V', '1593.973', 'IN3']
['UCPU1.AF52*', '4351.751', '3772.245', 'L', '1751.233', 'BOTTOM']
['TOTAL', '2', 'VIA(S)', '1751.233', 'mils']
['DDR1_DQ1_7']
['JDIMM1.170*', '3981.692', '2213.072', 'D', '0.000']
['VIA', '4000.245', '2338.330', 'V', '132.943', 'BOTTOM']
['VIA', '4300.155', '3610.605', 'V', '1578.602', 'IN3']
['UCPU1.AF53*', '4351.751', '3746.654', 'L', '1736.023', 'BOTTOM']
['TOTAL', '2', 'VIA(S)', '1736.023', 'mils']
['DDR1_DQ2_0']
['JDIMM1.232*', '4591.927', '2213.072', 'D', '0.000']
['VIA', '4610.810', '2320.680', 'V', '115.430', 'BOTTOM']
['VI

['FBA_D12']
['UV1.B11*', '8056.102', '4082.875', 'B', '0.000']
['VIA', '8070.866', '4068.111', 'V', '20.879', 'BOTTOM']
['VIA', '6923.111', '4034.015', 'V', '1875.787', 'IN1']
['UGPU1.C13*', '6907.088', '4050.785', 'B', '1899.194', 'BOTTOM']
['TOTAL', '2', 'VIA(S)', '1899.194', 'mils']
['FBA_D13']
['UV1.B12*', '8056.102', '4053.347', 'B', '0.000']
['VIA', '8041.338', '4038.583', 'V', '20.879', 'BOTTOM']
['VIA', '6922.355', '4001.178', 'V', '1843.389', 'IN1']
['UGPU1.B13*', '6907.088', '4019.295', 'B', '1867.830', 'BOTTOM']
['TOTAL', '2', 'VIA(S)', '1867.830', 'mils']
['FBA_D14']
['UGPU1.E13*', '6907.088', '4113.775', 'B', '0.000']
['VIA', '6891.625', '4098.575', 'V', '21.759', 'BOTTOM']
['VIA', '8159.449', '4038.583', 'V', '2241.986', 'IN1']
['UV1.E13*', '8144.685', '4023.819', 'B', '2262.865', 'BOTTOM']
['TOTAL', '2', 'VIA(S)', '2262.865', 'mils']
['FBA_D15']
['UV1.E12*', '8144.685', '4053.347', 'B', '0.000']
['VIA', '8129.921', '4068.111', 'V', '20.879', 'BOTTOM']
['VIA', '6892.323',

['PR1016.1', '5312.665', '6021.145', 'L', '32.069', 'TOP']
['VIA', '5306.500', '5981.675', 'V', '74.093', 'TOP']
['PU1001.9*', '5309.150', '5938.570', 'L', '118.296', 'TOP']
['TOTAL', '1', 'VIA(S)', '118.296', 'mils']
['N25087269']
['PR1024.2*', '5350.000', '5678.000', 'L', '0.000']
['PR1011.2', '5352.000', '5710.000', 'L', '32.828', 'TOP']
['PC1020.1', '5352.000', '5745.000', 'L', '67.828', 'TOP']
['PU1001.19*', '5340.650', '5822.430', 'L', '149.959', 'TOP']
['TOTAL', '0', 'VIA(S)', '149.959', 'mils']
['N25088422']
['PR1002.2*', '5457.140', '5770.190', 'L', '0.000']
['PC1003.2', '5420.000', '5768.000', 'L', '38.047', 'TOP']
['PU1001.20*', '5356.400', '5822.430', 'L', '131.541', 'TOP']
['TOTAL', '0', 'VIA(S)', '131.541', 'mils']
['N25160559']
['PR8040.1*', '3363.500', '5553.500', 'L', '0.000']
['PU0801.5*', '3288.015', '5552.895', 'L', '75.736', 'TOP']
['TOTAL', '0', 'VIA(S)', '75.736', 'mils']
['N25297774']
['PR2005.2*', '8331.925', '3524.450', 'L', '0.000']
['VIA', '7742.000', '5082.

['C6221.2*', '417.000', '4510.000', 'L', '0.000']
['R6225.1', '390.000', '4510.000', 'L', '33.627', 'TOP']
['D6204.1', '364.635', '4509.388', 'L', '65.366', 'TOP']
['VIA', '310.578', '4486.849', 'V', '128.759', 'TOP']
['VIA', '165.635', '4513.388', 'V', '294.697', 'TOP']
['JDOCK1.B10*', '134.764', '4522.084', 'L', '329.170', 'BOTTOM']
['TOTAL', '2', 'VIA(S)', '329.170', 'mils']
['TBTB_RX1P']
['R6217.1*', '515.000', '4484.302', 'L', '0.000']
['FL6204.1', '523.635', '4485.690', 'L', '8.746', 'TOP']
['VIA', '656.375', '4468.273', 'V', '151.535', 'TOP']
['U5501.J12*', '777.315', '4384.355', 'L', '307.235', 'TOP']
['TOTAL', '1', 'VIA(S)', '307.235', 'mils']
['TBTB_RX1P_C']
['C6222.1*', '440.000', '4485.000', 'L', '0.000']
['FL6204.2', '474.415', '4485.690', 'L', '40.757', 'TOP']
['R6217.2*', '479.000', '4484.302', 'L', '45.547', 'TOP']
['TOTAL', '0', 'VIA(S)', '45.547', 'mils']
['TBTB_RX1P_CONN']
['JDOCK1.B11*', '134.764', '4502.394', 'L', '0.000']
['VIA', '165.635', '4490.388', 'V', '35.84

['UV2.A10*', '8528.543', '5078.937', 'L', '3676.192', 'BOTTOM']
['C4570.1*', '8568.072', '5063.873', 'L', '3700.051', 'TOP']
['VIA', '8461.804', '3786.450', 'V', '3874.050', 'BOTTOM']
['VIA', '8665.787', '3015.113', 'V', '4771.962', 'IN1']
['VIA', '9735.000', '2005.000', 'V', '6439.337', 'BOTTOM']
['R81091.2*', '9697.000', '2005.000', 'L', '6477.337', 'TOP']
['TOTAL', '10', 'VIA(S)', '6477.337', 'mils']
['VCC1R8VIDEO_MEM_R']
['R5017.1*', '9640.000', '1955.000', 'L', '0.000']
['R81091.1', '9630.000', '2005.000', 'L', '54.142', 'TOP']
['VIA', '9665.000', '2005.000', 'V', '89.142', 'TOP']
['Q5006.1*', '9615.000', '1930.000', 'L', '184.853', 'BOTTOM']
['TOTAL', '1', 'VIA(S)', '184.853', 'mils']
['VCC33_18']
['C7608.1*', '1077.710', '1972.000', 'L', '0.000']
['U7601.18*', '1072.830', '2036.700', 'L', '66.721', 'TOP']
['TOTAL', '0', 'VIA(S)', '66.721', 'mils']
['VCC3B_CMOS']
['JLCD1.6*', '9676.580', '5886.220', 'L', '0.000']
['VIA', '9687.000', '5824.000', 'V', '66.536', 'BOTTOM']
['R5110.2*

In [ ]:
#step4: 分析txt
#修改df
print("step4: Parsing txt file...")
print("       Calculating... - 25%")
for i in netNameList:
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    
    # 填第一個元件的layer空格 
    # ex. TOP = TOP
    if len(indexList) > 1:
        dfSQSR.loc[indexList[0],"layer"] = dfSQSR.loc[indexList[1],"layer"]
    else:
        dfSQSR.loc[indexList[0],"layer"] = "TOP"
        
    #每個netname裡面找gap
    length = len(indexList)
    for index in indexList[::-1]:
        if length > 1:
            gap = dfSQSR.loc[index]["length"] - dfSQSR.loc[index-1]["length"]
            dfSQSR.loc[index,"gap"] = gap
        else:
            gap = 0
            dfSQSR.loc[index,"gap"] = gap
        length -= 1
print("       Calculating... - 50%")
#刪除多餘的VIA
deleteNullVIAROW(dfSQSR)  

In [18]:
#在summary建立branch path & branch length
for i in netNameList:
    print(i)
    indexList = dfSQSR[dfSQSR["net_name"] == f"{i}"].index.tolist()
    path = ""
    connIdxList = []
    
    for idx in indexList:
       
        if not re.findall("VIA", dfSQSR.loc[idx, "location"]): #找出connnector index 排出VIA, VIA(T)
            connIdxList.append(idx)
    
    length = len(connIdxList)
    
    #找出connector 在每條indexlist 裡面的index
    num = 0
    netIndex = dfsummary[dfsummary["net_name"] == f"{i}"].index.tolist()
    
    #把起始點&終點 加入summary
    start_end = f"{dfSQSR.loc[indexList[0], 'location']}:{dfSQSR.loc[indexList[-1], 'location']}"
    print("startEn", start_end)
    dfsummary.loc[netIndex, "start_end_path"] = start_end
    
    
    for idx in range(length - 1):
        num +=1 
        pathIdx = []
        pathIdx.append(indexList.index(connIdxList[idx]))
        pathIdx.append(indexList.index(connIdxList[idx + 1]))
               
        #抓出兩兩conn
        connector = ""
        for j in pathIdx:
            connector += dfSQSR.loc[indexList[j], "location"] + ":"
        
        
        connector = connector[:-1]
        print(connector)
        dfsummary.loc[netIndex, f"path{num}"] = connector
        
        #算出兩兩conn間的長度
        branchLen = 0
        for k in indexList[ pathIdx[0]+1 : pathIdx[1]+1]:
            branchLen += dfSQSR.loc[k, "gap"]

        dfsummary.loc[netIndex, f"length{num}"] = branchLen
        
print("       Calculating... - 75%")

0401
startEn JRJ1.A2:R7501.2
JRJ1.A2:C7501.1
C7501.1:R7501.2
0402
startEn JRJ1.B2:VIA
JRJ1.B2:R7502.2
7302
startEn R7310.2:U7301.33
R7310.2:U7301.33
7303
startEn U7301.35:VIA
U7301.35:R7311.2
7304
startEn Y7301.3:U7301.10
Y7301.3:C7314.2
C7314.2:U7301.10
7305
startEn R7314.2:U7301.30
R7314.2:U7301.30
7306
startEn R7315.2:U7301.12
R7315.2:U7301.12
7307
startEn Y7301.1:R7312.1
Y7301.1:C7313.2
C7313.2:R7312.1
7309
startEn U7301.6:R7306.1
U7301.6:R7306.1
7310
startEn R7308.1:U7301.1
R7308.1:U7301.1
7312
startEn R7312.2:U7301.9
R7312.2:U7301.9
7601
startEn U7601.28:R7616.2
U7601.28:R7616.2
P3VALW_FB
startEn PU0501.3:PC0515.1
PU0501.3:PC0515.1
P3VALW_OUT
startEn PU0501.4:PR0504.1
PU0501.4:PR0504.1
P3V_PWRGD
startEn PU0501.2:PR0501.2
PU0501.2:PR0501.2
P5VOUT
startEn PR0305.2:PU0301.10
PR0305.2:PR0307.2
PR0307.2:PR0303.1
PR0303.1:PC0305.1
PC0305.1:PU0301.10
P5V_VCC
startEn PU0301.9:PC0317.2
PU0301.9:PC0317.2
-5V_VCC_1
startEn PU0401.9:PC0412.2
PU0401.9:PC0412.2
-VL_3.3V
startEn PR0108.1:PR0306

startEn UEC1.74:R8556.1
UEC1.74:R8556.1
-PWRSHUTDOWN
startEn Q4001.1:UEC1.96
Q4001.1:PR0107.1
PR0107.1:PR0002.1
PR0002.1:PR0508.1
PR0508.1:PQ051.5
PQ051.5:PR0510.2
PR0510.2:UEC1.96
-PWRSWITCH
startEn JSW1.2:JDOCK1.11
JSW1.2:D5102.2
D5102.2:R5102.2
R5102.2:JPWR1.3
JPWR1.3:UEC1.93
UEC1.93:JDOCK1.11
-PWRSW_EC
startEn R8616.1:D6401.1
R8616.1:TP9807.1
TP9807.1:UEC1.109
UEC1.109:UCPU1.DK41
UCPU1.DK41:D6401.1
-RJ45_ACTIVITY
startEn U7301.27:U7401.15
U7301.27:U7401.15
-RJ45_ACTIVITY_SYS
startEn JRJ1.A1:U7401.19
JRJ1.A1:U7401.19
-RJ45_LINKUP
startEn U7401.16:UCPU1.DK31
U7401.16:R7302.1
R7302.1:U7301.26
U7301.26:UCPU1.DK31
-RJ45_LINKUP_SYS
startEn JRJ1.B1:U7401.20
JRJ1.B1:U7401.20
-RSMRST
startEn R1337.2:UCPU1.DM35
R1337.2:D1301.2
D1301.2:Q1304.3
Q1304.3:D1302.2
D1302.2:UCPU1.DM35
-RSMRST_EC
startEn UEC1.121:Q1303.1
UEC1.121:R1339.1
R1339.1:Q1303.1
-RTCRST
startEn C2002.2:UCPU1.DN37
C2002.2:R2002.2
R2002.2:JCMOS1.2
JCMOS1.2:UCPU1.DN37
-RT_B_RESET
startEn U5501.L11:U5901.9
U5501.L11:R5926.1
R5926

startEn C5317.2:U5302.39
C5317.2:U5302.39
DDIB_2P
startEn C5316.1:UCPU1.Y11
C5316.1:UCPU1.Y11
DDIB_2P_C
startEn U5302.38:C5316.2
U5302.38:C5316.2
DDIB_3N
startEn C5323.1:UCPU1.T11
C5323.1:UCPU1.T11
DDIB_3N_C
startEn C5323.2:U5302.48
C5323.2:U5302.48
DDIB_3P
startEn C5322.1:UCPU1.T12
C5322.1:UCPU1.T12
DDIB_3P_C
startEn C5322.2:U5302.47
C5322.2:U5302.47
DDIB_AUX_N
startEn TP0303.1:UCPU1.AD9
TP0303.1:UCPU1.AD9
DDIB_AUX_P
startEn UCPU1.AB9:TP0302.1
UCPU1.AB9:TP0302.1
DDIB_CTRLCLK
startEn R5325.1:UCPU1.DM29
R5325.1:R5319.1
R5319.1:R0305.2
R0305.2:UCPU1.DM29
DDIB_CTRLCLK_R
startEn R5319.2:U5302.34
R5319.2:U5302.34
DDIB_CTRLDATA
startEn R5320.1:UCPU1.DK27
R5320.1:R5326.1
R5326.1:R0307.2
R0307.2:UCPU1.DK27
DDIB_CTRLDATA_R
startEn U5302.33:R5320.2
U5302.33:R5320.2
DDR0_ACT_N
startEn JDIMM2.114:UCPU1.BT53
JDIMM2.114:UCPU1.BT53
DDR0_ALERT_N
startEn UCPU1.AU50:JDIMM2.116
UCPU1.AU50:JDIMM2.116
DDR0_BA0
startEn JDIMM2.150:UCPU1.BV44
JDIMM2.150:UCPU1.BV44
DDR0_BA1
startEn JDIMM2.145:UCPU1.CB42
JDIMM2

DDR1_DQ1_4
startEn JDIMM1.183:UCPU1.AF49
JDIMM1.183:UCPU1.AF49
DDR1_DQ1_5
startEn JDIMM1.174:UCPU1.AF50
JDIMM1.174:UCPU1.AF50
DDR1_DQ1_6
startEn JDIMM1.173:UCPU1.AF52
JDIMM1.173:UCPU1.AF52
DDR1_DQ1_7
startEn JDIMM1.170:UCPU1.AF53
JDIMM1.170:UCPU1.AF53
DDR1_DQ2_0
startEn JDIMM1.232:UCPU1.AV47
JDIMM1.232:UCPU1.AV47
DDR1_DQ2_1
startEn JDIMM1.249:UCPU1.AR47
JDIMM1.249:UCPU1.AR47
DDR1_DQ2_2
startEn JDIMM1.236:UCPU1.AV45
JDIMM1.236:UCPU1.AV45
DDR1_DQ2_3
startEn JDIMM1.245:UCPU1.AR45
JDIMM1.245:UCPU1.AR45
DDR1_DQ2_4
startEn JDIMM1.250:UCPU1.AV41
JDIMM1.250:UCPU1.AV41
DDR1_DQ2_5
startEn JDIMM1.237:UCPU1.AR42
JDIMM1.237:UCPU1.AR42
DDR1_DQ2_6
startEn JDIMM1.246:UCPU1.AV42
JDIMM1.246:UCPU1.AV42
DDR1_DQ2_7
startEn JDIMM1.233:UCPU1.AR41
JDIMM1.233:UCPU1.AR41
DDR1_DQ3_0
startEn JDIMM1.224:UCPU1.AL47
JDIMM1.224:UCPU1.AL47
DDR1_DQ3_1
startEn JDIMM1.228:UCPU1.AL45
JDIMM1.228:UCPU1.AL45
DDR1_DQ3_2
startEn JDIMM1.215:UCPU1.AJ47
JDIMM1.215:UCPU1.AJ47
DDR1_DQ3_3
startEn JDIMM1.211:UCPU1.AJ45
JDIMM1.211:UCP

R5760.1:R5759.1
EQC1_2
startEn R5812.1:U5801.10
R5812.1:R5817.1
R5817.1:U5801.10
EQD0_1
startEn U5701.14:R5751.1
U5701.14:R5752.1
R5752.1:R5751.1
EQD0_2
startEn R5830.1:U5801.14
R5830.1:R5832.1
R5832.1:U5801.14
EQD1_1
startEn U5701.18:R5750.1
U5701.18:R5749.1
R5749.1:R5750.1
EQD1_2
startEn U5801.18:R5826.1
U5801.18:R5829.1
R5829.1:R5826.1
ESPICLK_60M
startEn R8557.2:R0720.2
R8557.2:UEC1.2
UEC1.2:C0701.1
C0701.1:R0720.2
ESPI_CLK_DEBUG
startEn R8557.1:TP8502.1
R8557.1:TP8502.1
ESPI_IO0
startEn R0724.2:R8559.2
R0724.2:UEC1.126
UEC1.126:R8559.2
ESPI_IO1
startEn R8560.2:R0723.2
R8560.2:UEC1.127
UEC1.127:R0723.2
ESPI_IO2
startEn R8561.2:R0722.2
R8561.2:UEC1.128
UEC1.128:R0722.2
ESPI_IO3
startEn R8562.2:R0721.2
R8562.2:UEC1.1
UEC1.1:R0721.2
ESPI_IO0_DEBUG
startEn TP8504.1:R8559.1
TP8504.1:R8559.1
ESPI_IO0_R
startEn UCPU1.DP52:R0724.1
UCPU1.DP52:R0724.1
ESPI_IO1_DEBUG
startEn TP8505.1:R8560.1
TP8505.1:R8560.1
ESPI_IO1_R
startEn R0723.1:UCPU1.DP50
R0723.1:UCPU1.DP50
ESPI_IO2_DEBUG
startEn TP850

...

startEn C5711.1:VIA
C5711.1:UCPU1.Y7
UCPU1.Y7:TP9814.1
PCIE4_L0_TXN_C
startEn C5711.2:U5701.3
C5711.2:U5701.3
PCIE4_L0_TXN_CONN
startEn C6407.1:U5701.36
C6407.1:U5701.36
PCIE4_L0_TXN_CONN_C
startEn C6407.2:JSSD1.47
C6407.2:JSSD1.47
PCIE4_L0_TXP
startEn C5710.1:VIA
C5710.1:UCPU1.Y5
UCPU1.Y5:TP9813.1
PCIE4_L0_TXP_C
startEn C5710.2:U5701.2
C5710.2:U5701.2
PCIE4_L0_TXP_CONN
startEn C6408.1:U5701.37
C6408.1:U5701.37
PCIE4_L0_TXP_CONN_C
startEn C6408.2:JSSD1.49
C6408.2:JSSD1.49
PCIE4_L1_RXN
startEn C5709.2:VIA
C5709.2:UCPU1.T2
UCPU1.T2:TP9812.1
PCIE4_L1_RXN_C
startEn U5701.16:C5709.1
U5701.16:C5709.1
PCIE4_L1_RXN_CONN
startEn R5729.1:JSSD1.29
R5729.1:JSSD1.29
PCIE4_L1_RXN_CONN_R
startEn U5701.23:R5729.2
U5701.23:R5729.2
PCIE4_L1_RXP
startEn C5708.2:VIA
C5708.2:UCPU1.T1
UCPU1.T1:TP9811.1
PCIE4_L1_RXP_C
startEn U5701.15:C5708.1
U5701.15:C5708.1
PCIE4_L1_RXP_CONN
startEn R5728.1:JSSD1.31
R5728.1:JSSD1.31
PCIE4_L1_RXP_CONN_R
startEn U5701.24:R5728.2
U5701.24:R5728.2
PCIE4_L1_TXN
startEn C5715.1:

TP5504.1:U5502.6
RT_SPI_MISO
startEn U5501.B4:U5502.2
U5501.B4:TP5502.1
TP5502.1:R5517.1
R5517.1:U5502.2
RT_SPI_MOSI
startEn U5502.5:U5501.C6
U5502.5:TP5505.1
TP5505.1:U5501.C6
S3_1.2V
startEn PR0605.2:PU0601.10
PR0605.2:PR0606.1
PR0606.1:PC0617.1
PC0617.1:PR0610.1
PR0610.1:PU0601.10
S5_1.2V
startEn PU0601.9:PR0607.2
PU0601.9:PC0623.1
PC0623.1:PR0609.1
PR0609.1:PR0608.2
PR0608.2:PR0607.2
SA0_CHA_P
startEn JDIMM2.256:R3504.2
JDIMM2.256:R3508.2
R3508.2:R3504.2
SA0_CHB_P
startEn JDIMM1.256:VIA
JDIMM1.256:R3311.2
R3311.2:R3304.2
SA1_CHA_P
startEn JDIMM2.260:VIA
JDIMM2.260:R3505.2
R3505.2:R3509.2
SA1_CHB_P
startEn JDIMM1.260:R3312.1
JDIMM1.260:R3305.2
R3305.2:R3312.1
SA2_CHA_P
startEn JDIMM2.166:R3510.2
JDIMM2.166:R3506.2
R3506.2:R3510.2
SA2_CHB_P
startEn R3306.2:JDIMM1.166
R3306.2:R3313.2
R3313.2:JDIMM1.166
SATA1_DEVSLP
startEn JSSD1.38:UCPU1.DG8
JSSD1.38:UCPU1.DG8
SD_MMC_CD
startEn U7601.30:R7606.1
U7601.30:R7606.1
SD_MMC_CD_CONN
startEn R7606.2:JREAD1.9
R7606.2:C7703.1
C7703.1:JREAD1.9
S

startEn UCPU1.AF5:C6111.1
UCPU1.AF5:C6111.1
TCP1_AUX_DN_C
startEn C6111.2:R6134.1
C6111.2:R6109.2
R6109.2:U6101.25
U6101.25:R6134.1
TCP1_AUX_DP
startEn UCPU1.AF7:C6110.1
UCPU1.AF7:C6110.1
TCP1_AUX_DP_C
startEn R6110.2:R6133.1
R6110.2:C6110.2
C6110.2:U6101.24
U6101.24:R6133.1
TCP1_LSRX
startEn R0311.1:R0312.1
R0311.1:R0312.1
TCP1_RX0_DN
startEn R6142.1:UCPU1.AU2
R6142.1:C6115.1
C6115.1:UCPU1.AU2
TCP1_RX0_DN_C
startEn U6101.18:C6115.2
U6101.18:C6115.2
TCP1_RX0_DN_CO-LAY
startEn R6142.2:R6150.1
R6142.2:R6150.1
TCP1_RX0_DP
startEn R6141.1:UCPU1.AU1
R6141.1:C6114.1
C6114.1:UCPU1.AU1
TCP1_RX0_DP_C
startEn U6101.19:C6114.2
U6101.19:C6114.2
TCP1_RX0_DP_CO-LAY
startEn R6141.2:R6149.1
R6141.2:R6149.1
TCP1_RX1_DN
startEn UCPU1.AT1:C6107.1
UCPU1.AT1:R6136.1
R6136.1:C6107.1
TCP1_RX1_DN_C
startEn U6101.10:C6107.2
U6101.10:C6107.2
TCP1_RX1_DN_CO-LAY
startEn R6144.1:R6136.2
R6144.1:R6136.2
TCP1_RX1_DP
startEn UCPU1.AT2:R6135.1
UCPU1.AT2:C6106.1
C6106.1:R6135.1
TCP1_RX1_DP_C
startEn U6101.9:C6106.2
U61

startEn C6902.1:UCPU1.CW8
C6902.1:UCPU1.CW8
USB3P2_TXP_C
startEn C6902.2:R6902.1
C6902.2:FL6901.1
FL6901.1:R6902.1
USB3P2_TXP_CONN
startEn R6902.2:JUSB1.9
R6902.2:FL6901.2
FL6901.2:D6901.1
D6901.1:D6901.9
D6901.9:JUSB1.9
USBC_PD_CC1
startEn U6301.D3:U5901.6
U6301.D3:C5906.2
C5906.2:U5901.6
USBC_PD_CC2
startEn U5901.7:U6301.D4
U5901.7:C5907.2
C5907.2:U6301.D4
USBC_PD_CC1_CONN
startEn U6301.B2:JUSBC1.A5
U6301.B2:U6301.A2
U6301.A2:D6310.1
D6310.1:JUSBC1.A5
USBC_PD_CC2_CONN
startEn U6301.B3:JUSBC1.B5
U6301.B3:U6301.A3
U6301.A3:D6305.1
D6305.1:JUSBC1.B5
USBC_PD_GATE_VBUS
startEn U5901.18:R6003.1
U5901.18:R6003.1
USBC_PD_GATE_VSYS
startEn U5901.32:R6004.1
U5901.32:R6004.1
USBC_PD_HPD
startEn U5901.28:UCPU1.DJ47
U5901.28:U6101.32
U6101.32:R6112.2
R6112.2:R6111.2
R6111.2:UCPU1.DJ47
USBC_PD_RX0N
startEn R6315.1:R6150.2
R6315.1:FL6304.4
FL6304.4:R6123.2
R6123.2:R6150.2
USBC_PD_RX0N_C
startEn R6315.2:C6317.1
R6315.2:FL6304.3
FL6304.3:C6317.1
USBC_PD_RX0N_CONN
startEn JUSBC1.B10:C6317.2
JUSBC1.B10

VCCST_PWRGD
startEn UCPU1.BP8:VIA
UCPU1.BP8:R1333.2
VCCST_PWRGD_R
startEn R1332.2:Q1302.3
R1332.2:R1333.1
R1333.1:Q1302.3
VCC_SENSE
startEn R1407.2:PC0811.2
R1407.2:PU0801.28
PU0801.28:PR0846.2
PR0846.2:PC0811.2
VCORF
startEn C8511.1:UEC1.44
C8511.1:UEC1.44
VDD18_1
startEn UGPU1.G8:VIA
VDD18_2
startEn UGPU1.G9:VIA
VDDA
startEn PU0201.7:VIA
PU0201.7:PR0229.1
PR0229.1:PR0214.1
PR0214.1:PC0207.1
PC0207.1:PR0211.2
VESPI
startEn UEC1.4:R8514.2
UEC1.4:R8514.2
VGATE
startEn PU0801.10:R8520.1
PU0801.10:R1320.1
R1320.1:PR0843.2
PR0843.2:R8520.1
VGATE_R
startEn R8520.2:UEC1.124
R8520.2:UEC1.124
VGA_BLON
startEn R0303.2:UEC1.84
R0303.2:UCPU1.DN8
UCPU1.DN8:UEC1.84
VGA_CORE_PSI
startEn UGPU1.A4:PR1705.1
UGPU1.A4:PR1705.1
VIDBUF
startEn PU1701.6:PR1711.2
PU1701.6:PR1711.2
VIDEO_PWM_VID
startEn UGPU1.C6:PR1708.1
UGPU1.C6:PR1708.1
VID_VGA
startEn PR1708.2:PU1701.5
PR1708.2:PU1701.5
VNNEXT_1P05
startEn UCPU1.DD17:R1611.2
UCPU1.DD17:UCPU1.DD18
UCPU1.DD18:R1611.2
VREF06
startEn PR0807.2:PR0804.1
PR0807.2

In [ ]:
#調整dfsummary的順序
startEndColumn = dfsummary.pop(dfsummary.columns[2])
dfsummary.insert(1, startEndColumn.name, startEndColumn)
   

#轉成final SQS資料
column1 = []
column2 = []
for row in range(dfsummary.shape[0]):
    column1.append(dfsummary.loc[row,"net_name"])
    column1.append(dfsummary.loc[row,"start_end_path"])

    
    column2.append("")
    column2.append(dfsummary.loc[row,"total_length"])

    


    for num in range(branchPathNum(dfsummary)):

        column1.append(dfsummary.loc[row,f"path{num + 1}"])
        column2.append(dfsummary.loc[row,f"length{num + 1}"])
        
final = {
    "SQS"    : column1,
    "length" :column2
}
#轉成df 順便去掉空值
dfFinal = pd.DataFrame(final).dropna(axis=0)    
print("       Calculating... - 100%")
print("       Parse txt file. -DONE")
    
    
# step5: 儲存檔案
filename = re.findall(r"^\w*", filepath)[0]
write = pd.ExcelWriter(f'{filename}.xlsx')
save_excel(write, dfFinal, "final")
save_excel(write, dfsummary, "summary")
save_excel(write, dfSQS, "data base")
save_excel(write, dfSQSR, "SQS")
write.save()

print(f"step5: Save data to {filename}.xlsx. -DONE")